In [ ]:
# Import dependencies and set plotting styles

%matplotlib inline

In [ ]:
from matplotlib import style
style.use('fivethirtyeight')

import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# Create engine to hawaii.sqlite

engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# Reflect an existing database into a new model

Base = automap_base()

In [ ]:
# Reflect the tables

Base.prepare(engine, reflect=True) 

In [ ]:
# View all of the classes that automap found

Base.classes.keys() 

In [ ]:
# Save references to each table

Measurement = Base.classes.measurement

Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB

session = Session(engine) 

In [ ]:
# Look at first row of each object as dict

first_row_M = session.query(Measurement).first()
first_row_M.__dict__

In [ ]:
first_row_S = session.query(Station).first()
first_row_S.__dict__

# Exploratory Precipitation Analysis

In [ ]:
# Date conversion functions

def convert_from_iso(obj):    
    return dt.date.fromisoformat(obj)
        
def convert_to_iso(obj):    
    return dt.date.toisoformat(obj)             

In [ ]:
# Find the most recent date in the data set.

dates = session.query(Measurement.date)

max_date1 = convert_from_iso(max(dates)[0])
min_date1 = convert_from_iso(min(dates)[0])

print(f"Max date in dataset: {max_date1}")
print(f"Min date in dataset: {min_date1}")

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

from dateutil.relativedelta import relativedelta                 # thx to Tom! 

last_12_months = max_date1 - relativedelta(months=12)

print(last_12_months)

In [ ]:
# Calculate the date one year from the last date in data set.

plus_one_yr = max_date1 + relativedelta(years=1)

print(plus_one_yr)

In [ ]:
# Perform a query to retrieve the data and precipitation scores

precip = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= last_12_months).all()

# print(dict(precip))

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column

df = pd.DataFrame(data=precip)

df = df.set_index(keys=df["date"], drop=True)

df = df.drop(["date"], axis=1)

df = df.rename(columns={"prcp": "precipitation"})

# Sort the dataframe by date

df = df.sort_index(axis=0)

df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

df.plot(figsize=(10, 6), rot=90, xlabel="Date", ylabel="Inches")

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data

df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset

stations1 = session.query(Station.station).distinct().count()
print(stations1)

In [ ]:
stations2 = session.query(Measurement.station).distinct().count()
print(stations2)

In [ ]:
station_lst = session.query(Station.station).distinct().all()
# print(station_lst)

new_lst = [station[0] for station in station_lst]

print(f"STATIONS: \n\n{new_lst}")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

newer_lst = []

for station in new_lst:
    cnt = session.query(Measurement.station).filter(Measurement.station == station).count()
    newer_lst.append((cnt, station))
    
newer_lst = sorted(newer_lst, reverse=True)

for data in newer_lst:
    print(f"Station: {data[1]}")
    print(f"Row Count: {data[0]}\n")    


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

most_active_station = newer_lst[0][1]
most_active_station_rows = newer_lst[0][0]


print(f"Most active station: {most_active_station}")
print(f"Observation count: {most_active_station_rows}")


sel = [Measurement.tobs,
       func.min(Measurement.tobs),
       func.max(Measurement.tobs),
       func.avg(Measurement.tobs)
       ]

results = session.query(*sel).filter(Measurement.station == most_active_station).first()

print(f"\nMIN temp: {results[1]} degrees F")
print(f"MAX temp: {results[2]} degrees F")
print(f"AVG temp: {round(results[3], 2)} degrees F")

In [ ]:
results = session.query(*sel).filter(Measurement.station == most_active_station).all()

print(results)

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

dates = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == most_active_station)

# print(dates.all())

max_date2 = convert_from_iso(max(dates)[0])

print(max_date2)

last_12_months = max_date2 - relativedelta(months=12)

# print(last_12_months)

results = dates.filter(Measurement.date >= last_12_months).all()

# print(results)

df = pd.DataFrame(data=results)

df = df.set_index(keys="date")

df.plot.hist(bins=12)
plt.xlabel("Temperature (F)")
plt.ylabel("Frequency")
plt.figure(figsize=(8,6))

In [ ]:
# Calculate the date one year from the last date in data set.
# Note the use of max_date1 from earlier in the notebook

plus_one_yr = max_date1 + relativedelta(years=1)

print(plus_one_yr)

# Close session

In [ ]:
# Close Session
session.close()